---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
import heapq
def graph_identification():
    n = 0
    clus_cff = [nx.average_clustering(P1_Graphs[n]) for n in range(len(P1_Graphs))]
    ave_pth = [nx.average_shortest_path_length(P1_Graphs[n]) for n in range(len(P1_Graphs))]
    
# Average degree distribution of 100 nodes with nlargest degree in each graph. High value indicate Power Law distribution 
    deg_sum = [(sum(heapq.nlargest(100,(P1_Graphs[n]).degree().values()))) for n in range(len(P1_Graphs))]
    ave_deg_dist = [d/100 for d in deg_sum]
    
    df = pd.DataFrame()
    df['clus_coeff'] = pd.Series(clus_cff)
    df['ave_srt_pth'] =  pd.Series(ave_pth)
    df['ave_deg_dist'] = pd.Series(ave_deg_dist)
    df.loc[(df['ave_deg_dist'] > 14) & (df['clus_coeff'] < 0.1), 'G_type'] = 'PA'
    df.loc[(df['ave_deg_dist'] < 14) & (df['clus_coeff'] < 0.1), 'G_type'] = 'SW_H'
    df.loc[(df['ave_deg_dist'] < 14) & (df['clus_coeff'] > 0.1), 'G_type'] = 'SW_L'
    
    return df['G_type'].tolist()
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score as auc
def salary_predictions():
    df = pd.DataFrame(index = G.nodes())
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(G.degree())
    df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    df['Betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))
    h,a = nx.hits(G)
    df['Authority'] = pd.Series(a).round(5)
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    X_train = (df[df.ManagementSalary.notnull()])
    Y_train = X_train.pop('ManagementSalary').to_frame()
    X_test = (df[np.isnan(df.ManagementSalary)]).drop(['ManagementSalary'], axis = 1)
    
    #X_train, X_test, Y_train, Y_test = train_test_split(X_train1, Y_train1, test_size=0.3)
    grad_prams = {'n_estimators':500, 'min_samples_split':500, 'min_samples_leaf':5, 'max_depth':5, 'random_state':0,
                  'learning_rate':0.05}
    model = GradientBoostingClassifier(**grad_prams).fit(X_train, Y_train.values.ravel())
    #model = LogisticRegression(solver='liblinear', C=100, random_state=0).fit(X_train, Y_train.values.ravel())
    pred_prob = model.predict_proba(X_test)[:, 1]
    result = pd.Series(pred_prob.round(1), index = X_test.index, dtype = 'float64')
    #score = auc(Y_test, pred_prob)
    
    return result #score
salary_predictions()

1       0.0
2       1.0
5       1.0
8       0.1
14      0.0
18      0.0
27      0.0
30      0.7
31      0.4
34      0.0
37      0.0
40      0.0
45      0.0
54      0.3
55      0.4
60      0.1
62      1.0
65      1.0
77      0.1
79      0.0
97      0.0
101     0.0
103     0.5
108     0.0
113     0.1
122     0.0
141     0.3
142     1.0
144     0.0
145     0.3
       ... 
913     0.0
914     0.0
915     0.0
918     0.1
923     0.0
926     0.1
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.1
950     0.1
951     0.0
953     0.0
959     0.0
962     0.0
963     0.6
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

In [6]:
##For the salary predictions your AUC 0.9303307043033071

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    future_connections['Comn_nbrs'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['Jacc_coef'] = [x[2] for x in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Pref_atch'] = [x[2] for x in nx.preferential_attachment(G, future_connections.index)]
    
    X_train = (future_connections[future_connections['Future Connection'].notnull()])
    Y_train = X_train.pop('Future Connection').to_frame()
    
    X_test = (future_connections[np.isnan(future_connections['Future Connection'])]).drop(['Future Connection'], axis = 1)
    
    #X_train, X_test, Y_train, Y_test = train_test_split(X_train1, Y_train1, test_size=0.3)
    #model = LogisticRegression(solver='liblinear', C=100, random_state=0).fit(X_train, Y_train.values.ravel())
    grad_prams = {'n_estimators':500, 'min_samples_split':500, 'min_samples_leaf':5, 'max_depth':5, 'random_state':0,
                  'learning_rate':0.05}
    model = GradientBoostingClassifier(**grad_prams).fit(X_train, Y_train.values.ravel())
    pred_prob = model.predict_proba(X_test)[:, 1]
    result = pd.Series(pred_prob.round(2), index = X_test.index, dtype = 'float64')
    #score = auc(Y_test, pred_prob)
    
    return result #score
new_connections_predictions()

(107, 348)    0.03
(542, 751)    0.01
(20, 426)     0.58
(50, 989)     0.01
(942, 986)    0.01
(324, 857)    0.01
(13, 710)     0.11
(19, 271)     0.17
(319, 878)    0.01
(659, 707)    0.01
(49, 843)     0.01
(208, 893)    0.01
(377, 469)    0.01
(405, 999)    0.02
(129, 740)    0.03
(292, 618)    0.02
(239, 689)    0.01
(359, 373)    0.01
(53, 523)     0.03
(276, 984)    0.01
(202, 997)    0.01
(604, 619)    0.12
(270, 911)    0.01
(261, 481)    0.06
(200, 450)    0.88
(213, 634)    0.01
(644, 735)    0.13
(346, 553)    0.01
(521, 738)    0.01
(422, 953)    0.02
              ... 
(672, 848)    0.01
(28, 127)     0.98
(202, 661)    0.01
(54, 195)     1.00
(295, 864)    0.01
(814, 936)    0.01
(839, 874)    0.01
(139, 843)    0.01
(461, 544)    0.01
(68, 487)     0.01
(622, 932)    0.01
(504, 936)    0.02
(479, 528)    0.01
(186, 670)    0.01
(90, 395)     0.16
(329, 521)    0.03
(127, 218)    0.17
(463, 993)    0.01
(123, 142)    0.78
(764, 885)    0.01
(144, 824)    0.01
(742, 985)  

In [9]:
## Forthe new connections predictions your AUC 0.9088266675442648 